In [10]:
import numpy as np
from numba import njit, prange
import itertools as it

### Binary enumeration


The objective is to enumerate all of the possible combinations. To do this we  

In [11]:
#Compute all binary arrays of length n_x * n_y
def get_configurations(n_x,n_y):

    # Define the length of the binary numbers
    N = n_x * n_y

    # Define the number of binary numbers
    M = 2**N

    # Create an empty NumPy array of shape (N, M)
    binary_array = np.zeros((N, M), dtype='int32')

    # Fill each row with binary numbers
    for i in range(M):
        binary_representation = np.binary_repr(i, width=N)  # Get binary representation
        binary_array[:, i] = np.array(list(map(int, binary_representation)))  # Fill the row with binary values

    binary_array = binary_array.T
    return binary_array

n_x, n_y, n_z = 2,2,2

binary_array = get_configurations(n_x, n_y)

In [12]:
#find the bitwise and between each string and return a truth table where if X[i,j] = 1 then the ith and jth configurations are valid
# note that this only returns full occupation combinations
@njit(parallel = True)
def compute_truth_array(binary_array : np.array):
    #create empty array
    truth_array = np.zeros((binary_array.shape[0],binary_array.shape[0]))
    #parallel loop to find possible configurations
    for i in prange(binary_array.shape[0]):
        for j in range(i, binary_array.shape[0]):
            if (binary_array[i] & binary_array[j]).any() == 0:
                truth_array[i,j] = 1
            # truth_array[i,j] = np.sum(binary_array[i] & binary_array[j])
    return truth_array

truth_array = compute_truth_array(binary_array)

In [13]:
def get_possible_neighbours(x,y,z,n_x,n_y,n_z):
    neightbours = []
    z_hat = [z_ for z_ in range(n_z)]

    for z_ in z_hat:
        if x + 1 < n_x and x + 1 >= 0:
            neightbours.append((x + 1, y, z_))
        if x - 1 < n_x and x - 1 >= 0:
            neightbours.append((x - 1, y, z_))
        if y + 1 < n_y and y + 1 >= 0:
            neightbours.append((x, y + 1, z_))
        if y - 1 < n_y and y - 1 >= 0:
            neightbours.append((x, y - 1, z_))

    return neightbours

def possible_adj_matrix(c1,c2,n_x,n_y,n_z,K):
    c1,c2 = c1.reshape((n_x,n_y)),c2.reshape((n_x,n_y))
    X = np.zeros((n_x,n_y,n_z))
    X[:,:,0] = c1
    X[:,:,1] = c2
    A = np.zeros((n_x*n_y*n_z,n_x*n_y*n_z))
    for i in range(n_x):
        for j in range(n_y):
            for k in range(n_z):
                for i_,j_,k_ in get_possible_neighbours(i,j,k,n_x,n_y,n_z):
                    A[i + j*n_x + k * n_x**2, i_ + j_ * n_x + k_ * n_x**2] = X[i,j,k] * X[i_,j_,k_] * K[k,k_]
    return A

In [14]:
###

'''
given that I have, for each pair of sites a set of possible adjacency matrices, I need to store the following:

The configurations and the adjacencies with the energy of each

The energy of each bond

data will look like:

data = {'configuration' : {'site_1' : int, 'site_2' : int, 'Adjcency_mats' : [....], 'Energies' : [sum_of_mats for mats in Adjcency_mats.values()], 'entropy' : }}
'''

"\ngiven that I have, for each pair of sites a set of possible adjacency matrices, I need to store the following:\n\nThe configurations and the adjacencies with the energy of each\n\nThe energy of each bond\n\ndata will look like:\n\ndata = {'configuration' : {'site_1' : int, 'site_2' : int, 'Adjcency_mats' : [....], 'Energies' : [sum_of_mats for mats in Adjcency_mats.values()], 'entropy' : }}\n"

In [15]:
def compute_entropy(adjacency_matrices):
    entropies = []
    for adj_matrix in adjacency_matrices:
        # Compute probability distribution
        p_1 = np.count_nonzero(adj_matrix) / adj_matrix.size
        p_0 = 1 - p_1

        # Avoid log(0) by setting 0 * log(0) = 0
        if p_1 == 0 or p_0 == 0:
            entropy = 0
        else:
            # Compute Shannon entropy
            entropy = - (p_1 * np.log2(p_1) + p_0 * np.log2(p_0))

        entropies.append(entropy)

    # Return average entropy
    return np.mean(entropies)

def compute_entropy_with_weights(adjacency_matrices):
    entropies = []
    for adj_matrix in adjacency_matrices:
        # Shift weights to make them non-negative
        min_weight = np.min(adj_matrix)
        if min_weight < 0:
            adj_matrix -= min_weight

        # Normalize weights to probabilities
        adj_matrix /= np.sum(adj_matrix)

        # Compute Shannon entropy
        entropy = - np.sum(adj_matrix * np.log2(adj_matrix + np.finfo(float).eps))  # Add epsilon to avoid log(0)
        entropies.append(entropy)

    # Return average entropy
    return np.mean(entropies)

In [62]:
compute_entropy_with_weights(possible_adj_matrix(binary_array[9],binary_array[11],n_x,n_y,n_z,K))

/var/folders/jn/j9nggrw13g9gcrkh0hbk4mnc0000gn/T/ipykernel_44427/4169182392.py:29: RuntimeWarning: invalid value encountered in divide
  adj_matrix /= np.sum(adj_matrix)


nan

In [70]:
a = np.array([1,1,0,1])
b = np.array([0,0,1,0])

In [71]:
a.reshape((2,2)),b.reshape((2,2))

(array([[1, 1],
        [0, 1]]),
 array([[0, 0],
        [1, 0]]))

In [72]:
possible_adj_matrix(a,b,n_x,n_y,n_z,K)

array([[ 0.,  0.,  1.,  0.,  0., -1., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0., -0.],
       [ 1.,  0.,  0.,  1., -0.,  0.,  0., -0.],
       [ 0.,  0.,  1.,  0.,  0., -1., -0.,  0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0.]])

In [16]:
def count_ones(binary_string):
    count = 0
    for digit in binary_string:
        if digit == '1':
            count += 1
    return count

In [57]:
config_data = {}
K = np.array([[1,-1],[-1,0]])

for i,c1 in enumerate(binary_array):
    for j,c2 in enumerate(binary_array):
        if truth_array[i,j] == 1:
            adjacency_mats = possible_adj_matrix(c1,c2,n_x,n_y,n_z,K)
            config_data.update({
                f'{c1},{c2}' : {
                    'adj_mats' :adjacency_mats,
                    'Energies' : [np.sum(mat) for mat in adjacency_mats],
                    'Entropys' : compute_entropy_with_weights(possible_adj_matrix(c1,c2,n_x,n_y,n_z,K)),
                    'occupation_numbers' : [count_ones(c1),count_ones(c2)]
                                }
                }
                )

/var/folders/jn/j9nggrw13g9gcrkh0hbk4mnc0000gn/T/ipykernel_44427/4169182392.py:29: RuntimeWarning: invalid value encountered in divide
  adj_matrix /= np.sum(adj_matrix)


In [58]:
config_data.items()

dict_items([('[0 0 0 0],[0 0 0 0]', {'adj_mats': array([[ 0.,  0.,  0.,  0.,  0., -0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0.,  0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0.]]), 'Energies': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'Entropys': nan, 'occupation_numbers': [0, 0]}), ('[0 0 0 0],[0 0 0 1]', {'adj_mats': array([[ 0.,  0.,  0.,  0.,  0., -0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0.,  0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0.]]), 'Energies': [0.0, 0.0, 